In [2]:
import sys, argparse, logging
import pymssql
import _mssql
import datetime
from decimal import Decimal
import pymongo
from pymongo import MongoClient


from tmqr.settings import *

try:
    from tmqr.settings_local import *
except:
    pass



# SQL Server credentials
SQL_HOST = 'h9ggwlagd1.database.windows.net'
SQL_USER = 'modelread'
SQL_PASS = '4fSHRXwd4u'

In [3]:
sql_conn = pymssql.connect(SQL_HOST, SQL_USER + "@" + SQL_HOST, SQL_PASS, 'TMLDB')

In [4]:
def convert_dates(values):
    k,v = values
    if type(v) == datetime.date:
        return k, datetime.datetime.combine(
                v,
                datetime.datetime.min.time())
    if type(v) == Decimal:
        return k, float(v)
    else:
        return k, v

In [5]:
instruments = {}
colname = 'instruments'
qry = 'SELECT * FROM cqgdb.tbl{0}'.format(colname)
logging.debug(qry)
c2 = sql_conn.cursor(as_dict=True)
c2.execute(qry)
for row in c2:
    data = dict(map(convert_dates, row.items()))
    #print(data)
    instruments[data['idinstrument']] = data

In [6]:
instruments

{1.0: {'admcode': '17',
  'admexchangecode': '01',
  'admfuturepricefactor': 1.0,
  'admoptionftpfilestrikedisplay': 5.0,
  'admoptionpricefactor': 1.0,
  'commissionpercontract': 1.5,
  'cqgsymbol': 'USA',
  'customdayboundarytime': '1900-01-01 11:10:00',
  'datastart': '2008-08-06',
  'decisionoffsetminutes': 5,
  'description': 'US 30 YR Bond',
  'enabled': 1,
  'exchangesymbol': 'ZB',
  'exchangesymbolTT': 'ZB',
  'idAssetClass': 1,
  'idexchange': 1.0,
  'idinstrument': 1.0,
  'idinstrumentgroup': 2.0,
  'instrumentid_eod': 0,
  'instrumentsymbol_pre_eod': '',
  'instrumentsymboleod_eod': '',
  'limittickoffset': 10.0,
  'listedspread': 0,
  'margin': 2750.0,
  'modeled': 1,
  'notes': '',
  'optionadmstrikedisplay': 5.0,
  'optiondatamonthscollected': 36,
  'optionenabled': 2,
  'optionexchangesymbol': 'OZB',
  'optionexchangesymbolTT': 'OZB',
  'optionstart': '1900-01-01',
  'optionstrikedisplay': 100.0,
  'optionstrikedisplayTT': 1.0,
  'optionstrikeincrement': 1.0,
  'optionti

In [8]:
contracts = {}
colname = 'contracts'
#qry = 'SELECT TOP 10 * FROM cqgdb.tbl{0}'.format(colname)
qry = 'SELECT * FROM cqgdb.tbl{0}'.format(colname)
logging.debug(qry)
c2 = sql_conn.cursor(as_dict=True)
c2.execute(qry)
for row in c2:
    data = dict(map(convert_dates, row.items()))
    #print(data)
    tokens = data['contractname'].split('.')
    if '.' not in data['contractname'] or len(tokens) != 3:
        print('Wrong contract name: ' + data['contractname'])
        continue
    ctype, cmkt, cname = tokens
    
    idinstrument = data['idinstrument']
    
    if idinstrument not in instruments:
        print("idinstrument = {0} is not found for {1}".format(idinstrument,data['contractname']))
        continue
    
    underlying = instruments[idinstrument]['exchangesymbol']
    expiration = datetime.datetime.strptime(data['expirationdate'], '%Y-%m-%d')
    contract = '{0}.{1}{2}'.format(underlying, 
                                  data['month'],
                                  str(int(data['year']))[2:])
    ticker = '{0}.{1}.{2}.{3}'.format(cmkt, 
                                      ctype, 
                                      contract,
                                      expiration.strftime('%y%m%d')       
                                    )
    contracts[data['idcontract']] = {'tckr': ticker, 
                                     'contr': contract, 
                                     'type': ctype, 
                                     'underlying': '{0}.{1}'.format(cmkt, underlying),
                                     'instr': '{0}.{1}'.format(cmkt, underlying),
                                     'exp': expiration,
                                     'mkt': cmkt,
                                     'extra_data': {
                                            'month': data['month'],
                                            'monthint': data['monthint'],
                                            'year': data['year'],
                                            'name': data['contractname'],
                                            'sqlid': data['idcontract'],
                                        }
                                     }
    #print(ticker)
    #break

Wrong contract name: ED1982H
Wrong contract name: ED1982M
Wrong contract name: ED1982U
Wrong contract name: ED1982Z
Wrong contract name: ED1983H
Wrong contract name: ED1983M
Wrong contract name: ED1983U
Wrong contract name: ED1983Z
Wrong contract name: ED1984H
Wrong contract name: ED1984M
Wrong contract name: ED1984U
Wrong contract name: ED1984Z
Wrong contract name: ED1985H
Wrong contract name: ED1985M
Wrong contract name: ED1985U
Wrong contract name: ED1985Z
Wrong contract name: ED1986H
Wrong contract name: ED1986M
Wrong contract name: ED1986U
Wrong contract name: ED1986Z
Wrong contract name: ED1987H
Wrong contract name: ED1987M
Wrong contract name: ED1987U
Wrong contract name: ED1987Z
Wrong contract name: ED1988H
Wrong contract name: ED1988M
Wrong contract name: ED1988U
Wrong contract name: ED1988Z
Wrong contract name: ED1989H
Wrong contract name: ED1989M
Wrong contract name: ED1989U
Wrong contract name: ED1989Z
Wrong contract name: ED1990H
Wrong contract name: ED1990M
Wrong contract

In [9]:
# Init mongo asset index
client = MongoClient(MONGO_CONNSTR)
mongo_db = client[MONGO_DB]

In [10]:
# Storing futures
mongo_collection = mongo_db['asset_index']

mongo_collection.create_index([('tckr', pymongo.ASCENDING)], unique=True)

mongo_collection.create_index([('contr', pymongo.ASCENDING), 
                               ('mkt', pymongo.ASCENDING), 
                               ('type', pymongo.ASCENDING)])

cnt = 0
dup_cnt = 0
for c in contracts.values():
    try:
        mongo_collection.insert_one(c)
        #mongo_collection.replace_one({'tckr': c['tckr']}, c, upsert=True)
        cnt += 1
    except pymongo.errors.DuplicateKeyError:
        #print("Duplicated record: " + c['tckr'])
        dup_cnt += 1
        
print("Records added: {0} Duplicated: {1}".format(cnt, dup_cnt))       
    

Records added: 4320 Duplicated: 33


In [11]:
colname = 'options'
qry = 'SELECT * FROM cqgdb.tbl{0}'.format(colname)
logging.debug(qry)
c2 = sql_conn.cursor(as_dict=True)
c2.execute(qry)
cnt = 0
dup_cnt = 0
for row in c2:
    data = dict(map(convert_dates, row.items()))
    #print(data)
    ctype, cmkt, cname = data['optionname'].split('.')
    
    if data['idcontract'] not in contracts:
        continue
    
    underlying_dict = contracts[data['idcontract']]
    underlying = underlying_dict['tckr'].replace('US.', '').replace('.','-') #"{0}-{1}".format(underlying_dict['type'], underlying_dict['contr'])
    expiration = datetime.datetime.strptime(data['expirationdate'], '%Y-%m-%d')
    
    ticker = '{0}.{1}.{2}.{5}@{6}'.format(cmkt, 
                                     ctype, 
                                     underlying, 
                                     data['optionmonth'],
                                     str(int(data['optionyear']))[2:],
                                     expiration.strftime('%y%m%d'),
                                     data['strikeprice']   
                                    )
    opt_record = {'tckr': ticker, 
                 'type': ctype, 
                 'underlying': underlying_dict['tckr'],
                 'exp': expiration,
                 'instr': underlying_dict['instr'],
                 'mkt': cmkt,
                 'opttype': data['callorput'],
                 'strike': data['strikeprice'],
                 'optcode': data['optioncode'],
                 'extra_data': {
                        'month': data['optionmonth'],
                        'monthint': data['optionmonthint'],
                        'year': data['optionyear'],
                        'name': data['optionname'],
                        'sqlid': data['idoption'],
                    }
                 }
    #print(ticker)
    #break
    try:        
        mongo_collection.insert_one(opt_record)
        cnt += 1
    except pymongo.errors.DuplicateKeyError:
        dup_cnt += 1
        #print("Duplicated record: " + c['tckr'])

print("Records added: {0} Duplicated: {1}".format(cnt, dup_cnt))

Records added: 468138 Duplicated: 102


In [12]:
contracts

{1.0: {'_id': ObjectId('58ad3f7833b18260b69d9e2e'),
  'contr': 'CL.M83',
  'exp': datetime.datetime(1983, 5, 20, 0, 0),
  'extra_data': {'month': 'M',
   'monthint': 6,
   'name': 'F.US.CLEM83',
   'sqlid': 1.0,
   'year': 1983.0},
  'instr': 'US.CL',
  'mkt': 'US',
  'tckr': 'US.F.CL.M83.830520',
  'type': 'F',
  'underlying': 'US.CL'},
 2.0: {'_id': ObjectId('58ad3f7833b18260b69d9e2f'),
  'contr': 'CL.N83',
  'exp': datetime.datetime(1983, 6, 22, 0, 0),
  'extra_data': {'month': 'N',
   'monthint': 7,
   'name': 'F.US.CLEN83',
   'sqlid': 2.0,
   'year': 1983.0},
  'instr': 'US.CL',
  'mkt': 'US',
  'tckr': 'US.F.CL.N83.830622',
  'type': 'F',
  'underlying': 'US.CL'},
 3.0: {'_id': ObjectId('58ad3f7833b18260b69d9e30'),
  'contr': 'CL.Q83',
  'exp': datetime.datetime(1983, 7, 20, 0, 0),
  'extra_data': {'month': 'Q',
   'monthint': 8,
   'name': 'F.US.CLEQ83',
   'sqlid': 3.0,
   'year': 1983.0},
  'instr': 'US.CL',
  'mkt': 'US',
  'tckr': 'US.F.CL.Q83.830720',
  'type': 'F',
  'und

In [13]:
opt_record

{'_id': ObjectId('58ad42a333b18260b6a4d43e'),
 'exp': datetime.datetime(2018, 4, 25, 0, 0),
 'extra_data': {'month': 'K',
  'monthint': 5,
  'name': 'P.US.GCEK18860',
  'sqlid': 11965465.0,
  'year': 2018},
 'instr': 'US.GC',
 'mkt': 'US',
 'optcode': ' ',
 'opttype': 'P',
 'strike': 860.0,
 'tckr': 'US.P.F-GC-M18-180627.180425@860.0',
 'type': 'P',
 'underlying': 'US.F.GC.M18.180627'}

In [28]:
exchanges = {}
colname = 'exchange'
qry = 'SELECT * FROM cqgdb.tbl{0}'.format(colname)
logging.debug(qry)
c2 = sql_conn.cursor(as_dict=True)
c2.execute(qry)
for row in c2:
    exchanges[float(row['idexchange'])] = row
    print(row)

{'spanexchangesymbol': 'CBT', 'idexchange': Decimal('1'), 'tradingTechnologiesExchange': 'CME', 'tradingTechnologiesGateway': 'CME', 'exchange': 'CBOT', 'spanExchWebAPISymbol': 'CBT'}
{'spanexchangesymbol': 'CME', 'idexchange': Decimal('2'), 'tradingTechnologiesExchange': 'CME', 'tradingTechnologiesGateway': 'CME', 'exchange': 'CME', 'spanExchWebAPISymbol': 'CME'}
{'spanexchangesymbol': 'NYM', 'idexchange': Decimal('3'), 'tradingTechnologiesExchange': 'CME', 'tradingTechnologiesGateway': 'CME', 'exchange': 'NYMEX', 'spanExchWebAPISymbol': 'NYMEX'}
{'spanexchangesymbol': 'CMX', 'idexchange': Decimal('4'), 'tradingTechnologiesExchange': 'CME', 'tradingTechnologiesGateway': 'CME', 'exchange': 'COMEX', 'spanExchWebAPISymbol': ''}
{'spanexchangesymbol': 'IPE', 'idexchange': Decimal('5'), 'tradingTechnologiesExchange': 'ICE_IPE', 'tradingTechnologiesGateway': 'ICE_IPE', 'exchange': 'ICE', 'spanExchWebAPISymbol': ''}
{'spanexchangesymbol': '', 'idexchange': Decimal('6'), 'tradingTechnologiesE

In [36]:
EXCHANGE_NAMESPACE = 'US'

for iid, instr in instruments.items():
    #print(instr)
    res = {
        'tckr': "{0}.{1}".format(EXCHANGE_NAMESPACE, instr['exchangesymbol']),
        'ticksize': instr['ticksize'],
        'tickvalue': instr['tickvalue'],
        'mkt': EXCHANGE_NAMESPACE,
        'exchange': exchanges[instr['idexchange']]['exchange'],
        'description': instr['description'],
        'exchangesymbol': instr['exchangesymbol'],
        'extra_data': {          
        }
        
    }
    res['extra_data'].update(exchanges[instr['idexchange']])
    print(res)
    break
    

{'mkt': 'US', 'exchangesymbol': 'XAY', 'tickvalue': 10.0, 'description': 'Emini Consumer Discretionary', 'tckr': 'US.XAY', 'exchange': 'CME', 'extra_data': {'spanexchangesymbol': 'CME', 'idexchange': Decimal('2'), 'tradingTechnologiesExchange': 'CME', 'tradingTechnologiesGateway': 'CME', 'exchange': 'CME', 'spanExchWebAPISymbol': 'CME'}, 'ticksize': 0.1}
